In [1]:
import os


In [11]:
os.chdir("../")

In [18]:
%pwd

'/Users/cairo-lopes/Documents/code/python/End-to-end-Machine-Learning-Project-with-MLflow'

In [25]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    

In [26]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [27]:
read_yaml(CONFIG_FILE_PATH)

[2023-11-09 14:30:53,778: INFO: common: yaml file: config/config.yaml loaded successfully]


ConfigBox({'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://github.com/entbappy/Branching-tutorial/raw/master/winequality-data.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}})

In [28]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath: Path = CONFIG_FILE_PATH,
            params_filepath: Path = PARAMS_FILE_PATH,
            schema_filepath: Path = SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

In [29]:
import os
import urllib.request as request
import zipfile
from mlProject import logger
from mlProject.utils.common import get_size

In [34]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig) -> None:
        self.config = config
    
    def download_data(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file)
            
            logger.info(f"{filename} downloaded! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip_file_path: str
        extracts the zip file into the data directory
        Function returns none
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        logger.info(f"File extraction is done")

In [35]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_data()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-11-09 14:33:08,439: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-11-09 14:33:08,442: INFO: common: yaml file: params.yaml loaded successfully]
[2023-11-09 14:33:08,444: INFO: common: yaml file: params.yaml loaded successfully]
[2023-11-09 14:33:08,445: INFO: common: created directory at: artifacts]
[2023-11-09 14:33:08,445: INFO: common: created directory at: artifacts/data_ingestion]
[2023-11-09 14:33:08,446: INFO: 889200314: File already exists of size: ~ 23 KB]
[2023-11-09 14:33:08,450: INFO: 889200314: File extraction is done]
